# Interpolation of f1 CAFE hybrid levels to isobaric levels
### NOTE: must run `ulimit -s unlimited` prior to running this notebook

https://pythonadventures.wordpress.com/2014/01/08/capture-the-exit-code-the-stdout-and-the-stderr-of-an-external-command/

In [6]:
import glob
import subprocess
import itertools
#import xarray as xr
import numpy as np
import time

## Location of cafe grid info

In [7]:
cafe_grid_info = '/OSM/CBR/OA_DCFP/work/squ027/squire_scratch/projects/cafe_hybrid_to_isobaric/shared/cafe_grid_info.nc3.nc '

## Get list of files to convert

In [12]:
years = np.arange(2003,2004)
months = np.arange(1,2)
ensembles = np.arange(1,2)
folder = '/OSM/CBR/OA_DCFP/data1/model_output/CAFE/forecasts/f1/hybrid/'
files = 'atmos_daily*01.nc'

filelist = []
for year, month, ensemble in itertools.product(years, months, ensembles):
    filelist.append(sorted(glob.glob(folder + 'yr' + str(year) 
                                            + '/mn' + str(month) 
                                            + '/OUTPUT.' + str(ensemble) 
                                            + '/' + files)))
    
filelist = list(itertools.chain.from_iterable(filelist))

print(filelist)

['/OSM/CBR/OA_DCFP/data1/model_output/CAFE/forecasts/f1/hybrid/yr2003/mn1/OUTPUT.1/atmos_daily_2003_01_01.nc', '/OSM/CBR/OA_DCFP/data1/model_output/CAFE/forecasts/f1/hybrid/yr2003/mn1/OUTPUT.1/atmos_daily_2004_01_01.nc', '/OSM/CBR/OA_DCFP/data1/model_output/CAFE/forecasts/f1/hybrid/yr2003/mn1/OUTPUT.1/atmos_daily_2005_01_01.nc', '/OSM/CBR/OA_DCFP/data1/model_output/CAFE/forecasts/f1/hybrid/yr2003/mn1/OUTPUT.1/atmos_daily_2006_01_01.nc', '/OSM/CBR/OA_DCFP/data1/model_output/CAFE/forecasts/f1/hybrid/yr2003/mn1/OUTPUT.1/atmos_daily_2007_01_01.nc', '/OSM/CBR/OA_DCFP/data1/model_output/CAFE/forecasts/f1/hybrid/yr2003/mn1/OUTPUT.1/atmos_daily_2008_01_01.nc']


## Levels to interpolate to

In [13]:
levels = [20000, 30000, 40000, 45000, 50000, 55000, 60000, 70000, 85000, 100000]

## Loop over each file and interpolate

In [34]:
import inspect
import os
import shutil

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

rundir='/OSM/CBR/OA_DCFP/work/col414/cafepp'

print('Current Working Directory=',os.getcwd())

#odir='/OSM/CBR/OA_DCFP/work/col414/cafepp'
tmpdir='/OSM/CBR/OA_DCFP/data1/col414'

os.chdir(rundir)

print('Current Working Directory=',os.getcwd())

for file in filelist:
    tmpfile=tmpdir+'/'+file.split('/')[-1]
    print(file)
    print(tmpfile)
    

#     shutil.copyfile(file,tmpfile)
    #print(file.split('/')[-1])
    # Add ak and bk fields to atmospheric output files -----
#     command = '/apps/nco/4.4.4/bin/ncks -A -v ak,bk,zsurf ' + cafe_grid_info + newfile
#     print(command)
#     out = subprocess.call(command, shell=True)
#     if (out != 0) & (out != 1):
#         print('Failed to add alphas and betas:')
#         print(file, '\n')
        
        
    #time.sleep(1.0) 
    
    # Rename ak with pk as required by PLEV executable -----
#     command = '/apps/nco/4.4.4/bin/ncrename -v ak,pk ' + tmpfile
#     print(command)
#     out = subprocess.call(command, shell=True)
#     if (out != 0) & (out != 1):
#         print('Failed to rename alpha:')
#         print(file, '\n')
        
    outfile=tmpfile.split('.nc')[0]+'.plevel.nc'
    
    print(outfile)
    
    if(os.path.exists(outfile)):
       os.remove(outfile)
    
    # Perform the interpolation -----
    command = '/OSM/CBR/OA_DCFP/work/squ027/squire_scratch/projects/cafe_hybrid_to_isobaric/test/plevel.sh -a -i ' + tmpfile + \
              ' -o ' + outfile + \
              ' -p ' + '"' + ' '.join(map(str, levels)) + '"' + \
              ' hght'
    print(command)
       
    out = subprocess.call(command, shell=True)
#     out = subprocess.run(command, shell=True, capture_output=True)
#     out = subprocess.run(command, shell=True, stderr=subprocess.STDOUT)
       
    raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

    if (out != 0) & (out != 1):
        print(f'Regridding failed, error {out}:')
        print(file, '\n')
    
    #time.sleep(1.0) 
    
    # Convert to netcdf-4 -----
    command = 'ncks -4 -L 1 ' + file[:file.index('.nc')] + '.tmp' + file[file.index('.nc'):] + ' ' \
                              + file[:file.index('.nc')] + '.plevel' + file[file.index('.nc'):]
    out = subprocess.call(command, shell=True)
    if (out != 0) & (out != 1):
        print('Failed to convert to netcdf-4:')
        print(file, '\n')
    
    #time.sleep(1.0) 
        
    # Delete temporary file -----
    command = 'rm -r ' + file[:file.index('.nc')] + '.tmp' + file[file.index('.nc'):]
    out = subprocess.call(command, shell=True)

Current Working Directory= /OSM/CBR/OA_DCFP/work/col414/cafepp
Current Working Directory= /OSM/CBR/OA_DCFP/work/col414/cafepp
/OSM/CBR/OA_DCFP/data1/model_output/CAFE/forecasts/f1/hybrid/yr2003/mn1/OUTPUT.1/atmos_daily_2003_01_01.nc
/OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.nc
/OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.plevel.nc
/OSM/CBR/OA_DCFP/work/squ027/squire_scratch/projects/cafe_hybrid_to_isobaric/test/plevel.sh -a -i /OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.nc -o /OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.plevel.nc -p "20000 30000 40000 45000 50000 55000 60000 70000 85000 100000" hght


SystemExit: STOP!:jupyter_notebook line number: 60

/OSM/CBR/OA_DCFP/apps/col414/anaconda3/envs/FERRET/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [51]:
import sys
import shlex
from subprocess import Popen, PIPE

command='/OSM/CBR/OA_DCFP/work/col414/cafepp/plevel_col414.bash -a -i /OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.nc -o /OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.plevel.nc'
print(command)

def get_exitcode_stdout_stderr(cmd):
    """
    Execute the external command and get its exitcode, stdout and stderr.
    """
    args = shlex.split(cmd)
 
    proc = Popen(args, stdout=PIPE, stderr=PIPE)
    out, err = proc.communicate()
    exitcode = proc.returncode
    #
    return exitcode, out, err

/OSM/CBR/OA_DCFP/work/col414/cafepp/plevel_col414.bash -a -i /OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.nc -o /OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.plevel.nc


In [59]:
print('Current Working Directory=',os.getcwd())

exitcode, out, err = get_exitcode_stdout_stderr(command)

# status = subprocess.call(command, shell=True)

# print(status)

Current Working Directory= /OSM/CBR/OA_DCFP/work/col414/cafepp


In [60]:
print(exitcode)

0


In [61]:
print(out)

b'hello\nitime =       1\nitime =       2\nitime =       3\nitime =       4\nitime =       5\nitime =       6\nitime =       7\nitime =       8\nitime =       9\nitime =      10\nitime =      11\nitime =      12\nitime =      13\nitime =      14\nitime =      15\nitime =      16\nitime =      17\nitime =      18\nitime =      19\nitime =      20\nitime =      21\nitime =      22\nitime =      23\nitime =      24\nitime =      25\nitime =      26\nitime =      27\nitime =      28\nitime =      29\nitime =      30\nitime =      31\nitime =      32\nitime =      33\nitime =      34\nitime =      35\nitime =      36\nitime =      37\nitime =      38\nitime =      39\nitime =      40\nitime =      41\nitime =      42\nitime =      43\nitime =      44\nitime =      45\nitime =      46\nitime =      47\nitime =      48\nitime =      49\nitime =      50\nitime =      51\nitime =      52\nitime =      53\nitime =      54\nitime =      55\nitime =      56\nitime =      57\nitime =      58\nitime

In [55]:
print(err)

b''
